## Convolutional Neural Networks
CNNs are great for problems that require classification (and sometimes regression) from visual data. CNNs are useful because *they* find the features themselves as opposed to us determining the features to use. We just give the CNN a label and data that is somewhat visual and let it go.
The general flow for this notebook will be:
1. Explore tools from PyTorch that allow for the import/transformation of different types of visual data
2. Importing data image data from a clothing database
3. Design a Multi-Class NN model that will be used to classify the different types of clothing in the images
4. Examine the results and then create a CNN model and compare the performance to the MCNN
5. Save the weights from model with the best results so that it can be used elsewhere

In [ ]:
# Lets start by importing everything we will need

import torch
from torch import nn

import torchvision
from torchvision import datasets # contains pre-built datasets that can be used to test models
from torchvision.transforms import ToTensor # contains useful functions that can transfrom common images formats to tensors

import matplotlib.pyplot as plt
print(f'PyTorch Version: {torch.__version__}, Torchvision Version: {torchvision.__version__}')

In [ ]:
# Lets get our training and testing data. Turns out torchvision has many built-in datasets already.
# We will be using the FashionMNIST dataset for this classification problem. It's basically the fashion
# version of the original MNIST dataset that used number. There are 10 classes, but they're clothes, not numbers.

DATA_DIR = "../../data/"

# Many of the datasets in this module have the same arguments.
train_data = datasets.FashionMNIST(
    DATA_DIR,
    train=True,
    transform=ToTensor(),
    target_transform=None,
    download=True
)

test_data = datasets.FashionMNIST(
    DATA_DIR,
    train=False,
    transform=ToTensor(),
    target_transform=None,
    download=True
)

In [ ]:
# With the training data donwloaded, lets explore
print(type(train_data))

# Lets get the first image. label combo from the test data
image, label = test_data[0]
print(f'Label Type: {type(label)}, Label: {label}')
print(f'Image Type: {type(image)}, Image Shape: {image.shape}')

#### So it looks like the image is a 3D tensor and the label is just an integer giving the class. The 1 in the first dimension of the image shows that it's just a greyscale image. This implies that the values in the 28x28 2D tensor just represent the intensity of the the pixel. If this were a color image, there would be 3 "channels" representing the intensity of RGB respectively. 

In [ ]:
# The dataset object has useful properties that allow you to view different aspects of the dataset.
print(f'Training size: {len(train_data)}, Test size: {len(test_data)}, Test/Train Ratio: {len(test_data)/len(train_data):.2f}')
print(train_data.classes)

In [ ]:
# Now lets actually look at some of the images/labels in the test dataset
fig = plt.figure(figsize=(9,9))
plt_rows = 4
plt_cols = 4
torch.manual_seed(42)
for sub in range(1, plt_rows * plt_cols + 1):
    index = torch.randint(0, len(train_data) - 1, size=[1]).item() # Get a random number (in range of the train data)
    img, label = train_data[index] # Pull a random test sample and it's label from the training data
    fig.add_subplot(plt_rows, plt_cols, sub)
    plt.imshow(img.squeeze(), cmap='gray') # Show the image in the subplot. Squeezing out the first dimension, not needed for 2D greyscale
    plt.title(train_data.classes[label])
    plt.axis(False)

#### As seen above, we have quite a few different images corresponding to the different classes in the dataset! The next step is create a Dataloader object for input into the model we plan to create. This seems like an arbitrary step, why do we need to transform the data into some special object as opposed to just using it as it exists in tensor form? The main reason is due to __Batching__. In most real world scenarios, datasets will be huge. So far, we've been doing a form of training called __Batch Gradient Descent__ where *backprop is done on the entire training set for each epoch*. This is computationally possible with small datasets, but with larger datasets, this would be computationally expensive (thus actually expensive in time and cost). The method we'll be adopting is the use of mini-batching; grouping a very small subset of the entire training set to be used for one backprop step. Instead of the loss from the WHOLE dataset (Batch) or each individual sample (Stochastic) being used to change the weights in the model, the *average* loss of the mini-batch is used. This is effective in reducing the computational load and to prevent issues like overfitting. Moving our data to a Dataloader object allows for us to easily change the batch size and to allow for shuffling the data around for each epoch (epoch # = mini-batch #).

In [ ]:
# Create the dataloaders obj for both training and test
from torch.utils.data import DataLoader
BATCH_SIZE = 32
train_dloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_dloader = DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Now lets inpect the objects. Much of the logic to batch and match labels to input data is done for us in the Dataloader obj
print(f'Type: {type(train_dloader)}')
train_batch_img, train_batch_labels = next(iter(train_dloader))
print(f'Batch Img: {train_batch_img.shape}, Batch Labels: {train_batch_labels.shape}') # So the dloader is basically a list of all the batches and their corressponding labels

### Model Creation
With all the data import and preparation done, we can now create a baseline (very simple) model. Will start with a basic 2 layer network, one hidden layer, with a flattening layer as the input layer.

In [ ]:
INPUT_DIMS = train_data[0][0].shape[1]*train_data[0][0].shape[2] # our images are 28*28 and will be flattened out before hitting the NN
HIDDEN_UNITS = 10
OUTPUT_DIMS = len(train_data.classes)

class FashionMNISTModel0(nn.Module):
    def __init__(self,
                 in_shape: int,
                 out_shape: int,
                 hidden_units: int) -> None:
        super().__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=in_shape, out_features=hidden_units, bias=True),
            nn.Linear(in_features=hidden_units, out_features=out_shape, bias=True)
        )
    def forward(self, x):
        return self.network(x)

fmnist_model_0 = FashionMNISTModel0(in_shape=INPUT_DIMS,
                               out_shape=OUTPUT_DIMS,
                               hidden_units=HIDDEN_UNITS)
print(fmnist_model_0)



In [ ]:
# Need to once again define the optimizer/loss function combo
optimizer = torch.optim.SGD(params=fmnist_model_0.parameters(), lr=0.1)
loss_fn = nn.CrossEntropyLoss() # 'mean' reduction takes all the loss values from the batch and averages them to get the loss

In [ ]:
# Now it's training loop time. Since we're using a slightly different model architecture with mini-batches,
# will write out all steps. Mini-batches require a nested loop for each epoch.
from timeit import default_timer as timer # boilerplate timer functionality
from tqdm.auto import tqdm # text-based progress bar
from torchmetrics import Accuracy

# Now lets create a quick little function that gives the run time of the loop
total_time = lambda start_time, stop_time: stop_time - start_time

torch.manual_seed(42)
EPOCHS = 2
train_size = len(train_dloader.dataset) # number of samples in the train dataset
test_size = len(test_dloader.dataset) # number of samples in test dataset
num_batches_train = train_size/BATCH_SIZE
num_batches_test = test_size/BATCH_SIZE
acc = Accuracy(task='multiclass', num_classes=len(test_data.classes))

train_time_mnn_start = timer()
for epoch in tqdm(range(EPOCHS)):
    print(f'Epoch: {epoch}\n-----------')

    # Training Steps
    train_loss = 0 # need to reset loss every epoch
    for batch, (X_train, y_train) in enumerate(train_dloader): # each batch has 32 data/labels, create object -> (batch, (X_train, y_train))
        fmnist_model_0.train()
        
        y_logits = fmnist_model_0(X_train) # Like before, need to get model's predictions (in logits)
        loss = loss_fn(y_logits, y_train) # calculate loss for this batch
        train_loss += loss # add loss from this batch (mean loss of 32 samples) to total loss for the epoch (sum of all batch loss)

        # backprop step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # We want to see some updates within an epoch
        print(f'Batches processed: {batch + 1}/{int(num_batches_train)}, Samples processed: {(batch + 1) * BATCH_SIZE}/{train_size}', end='\r')
    
    # Now we want to find the AVERAGE loss of all the batches
    train_loss /= num_batches_train

    # Test Steps; mimick the training steps without backprop.
    # Only care about the epoch level values for test, no intermediate
    # updates necessary.
    test_loss = 0
    test_acc = 0

    fmnist_model_0.eval()
    with torch.inference_mode():
        for X_test, y_test in test_dloader:
            test_logits = fmnist_model_0(X_test)
            test_loss += loss_fn(test_logits, y_test) # Note, no need for batch loss variable since we don't care about per batch backprop when testing
            test_acc += acc(test_logits, y_test) # acc expects input tensors denoting labels, need to convert from logits

        # Get AVERAGE loss and accuracy off all test batches
        test_loss /= num_batches_test 
        test_acc /= num_batches_test 

        # print out test loss and test accuracy
    print('\n-----------')
    print(f'Mean Test Loss: {test_loss:.4f}')
    print(f'Mean Test Accuracy: {test_acc * 100:.2f}%')
    print('-----------\n')
train_time_mnn_end = timer()
print(f'Total time to train: {total_time(train_time_mnn_start, train_time_mnn_end):.2f}s')

#### Seems like the baseline model is doing a decent job, but lets see if we can make it any better by adding non-linearity to the model architecture

In [ ]:
# Since the train/test loop has gotten more complex, lets turn the steps into functions
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn):

    # Training Steps
    loss, acc = 0, 0 # need to reset loss every epoch
    for batch, (X, y) in enumerate(data_loader): # each batch has 32 data/labels, create object -> (batch, (X, y))
        model.train()
        y_pred = model(X) # Like before, need to get model's predictions (in logits)
        loss = loss_fn(y_pred, y) # calculate loss for this batch
        loss += loss # add loss from this batch (mean loss of 32 samples) to total loss for the epoch (sum of all batch loss)
        acc += accuracy_fn(y_pred.argmax(dim=1), y)

        # backprop step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # We want to see some updates within an epoch
        print(f'Batches processed: {batch + 1}/{len(data_loader)}, Samples processed: {(batch + 1) * data_loader.batch_size}/{len(data_loader.dataset)}', end='\r')
    
    # Now we want to find the AVERAGE loss and accuracy of all the batches
    loss /= len(data_loader)
    acc /= len(data_loader)
    print('\n-----------')
    print(f'Mean Train Loss: {loss:.4f}, Mean Train Accuracy: {acc * 100:.2f}%')

def test_step(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn):

    # Test Steps
    model.eval()
    loss, acc = 0, 0 # need to reset loss every epoch
    with torch.inference_mode():
        for X, y in data_loader: # each batch has 32 data/labels, create object -> (batch, (X_train, y_train))
            y_pred = model(X) # Like before, need to get model's predictions (in logits)
            loss = loss_fn(y_pred, y) # calculate loss for this batch
            loss += loss # add loss from this batch (mean loss of 32 samples) to total loss for the epoch (sum of all batch loss)
            acc += accuracy_fn(y_pred.argmax(dim=1), y)

        # Now we want to find the AVERAGE loss and accuracy of all the batches
        loss /= len(data_loader)
        acc /= len(data_loader)
    print(f'Mean Test Loss: {loss:.4f}, Mean Test Accuracy: {acc * 100:.2f}%')
    print('-----------\n')

In [ ]:
# Lets try out new functions out in an identical training loop as before
# Now it's training loop time. Since we're using a slightly different model architecture with mini-batches,
# will write out all steps. Mini-batches require a nested loop for each epoch.

from helper_functions import model_accuracy
torch.manual_seed(42)
EPOCHS = 2
train_time_mnn_start = timer()
for epoch in tqdm(range(EPOCHS)):
    print(f'Epoch: {epoch}\n-----------')
    train_step(
        fmnist_model_0,
        train_dloader,
        loss_fn,
        optimizer,
        model_accuracy
    )
    test_step(
        fmnist_model_0,
        test_dloader,
        loss_fn,
        model_accuracy
    )
train_time_mnn_end = timer()
print(f'Total time to train: {total_time(train_time_mnn_start, train_time_mnn_end):.2f}s')

In [ ]:
# With this simplified training/testing loop code, lets define a new model with non-linearities and try to test it again
INPUT_DIMS = train_data[0][0].shape[1]*train_data[0][0].shape[2] # our images are 28*28 and will be flattened out before hitting the NN
HIDDEN_UNITS = 10
OUTPUT_DIMS = len(train_data.classes)

class FashionMNISTModel1(nn.Module):
    def __init__(self,
                 in_shape: int,
                 out_shape: int,
                 hidden_units: int) -> None:
        super().__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=in_shape, out_features=hidden_units, bias=True),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=out_shape, bias=True),
            nn.ReLU()
        )
    def forward(self, x):
        return self.network(x)

fmnist_model_1 = FashionMNISTModel1(in_shape=INPUT_DIMS,
                               out_shape=OUTPUT_DIMS,
                               hidden_units=HIDDEN_UNITS)
print(fmnist_model_1)

In [ ]:
torch.manual_seed(42)
optimizer_v1 = torch.optim.SGD(params=fmnist_model_1.parameters(), lr=0.1)
loss_fn = nn.CrossEntropyLoss() # 'mean' reduction takes all the loss values from the batch and averages them to get the loss

EPOCHS = 2
train_time_mnn_start = timer()
for epoch in tqdm(range(EPOCHS)):
    print(f'Epoch: {epoch}\n-----------')
    train_step(
        fmnist_model_1,
        train_dloader,
        loss_fn,
        optimizer_v1,
        model_accuracy
    )
    test_step(
        fmnist_model_1,
        test_dloader,
        loss_fn,
        model_accuracy
    )
train_time_mnn_end = timer()
print(f'Total time to train: {total_time(train_time_mnn_start, train_time_mnn_end):.2f}s')

### CNN
Now with the multi-class NN complete, lets update the model to use a CNN! We'll be using the model architecture called "TinyVGG" from the [CNN Explainer](https://poloclub.github.io/cnn-explainer/) website. Let's now create some test data to explore what the different types of layers used in the model do.

In [ ]:
import torch as torch
from torch import nn

In [ ]:
# Lets define some constants and then look at nn.Conv2d
BATCH_SIZE = 32
CHANNELS = 3
SAMPLE_HEIGHT = 64
SAMPLE_WIDTH = 64
SEED = 42

torch.manual_seed(SEED)
train_sample = torch.randn(size=(BATCH_SIZE, CHANNELS, SAMPLE_HEIGHT, SAMPLE_WIDTH))
print(train_sample.dtype)
print(f'First channel of the first training sample:\n {train_sample[0][0]}')
print(f'Shape of the first training sample: {train_sample[0][0].shape}')

In [ ]:
# Now lets create a Conv2d layer
conv_layer = nn.Conv2d(in_channels=3, # num channels in the input data
                       out_channels=10, # num channels (filters) in the output data
                       kernel_size=3, # size of sliding window that does the convolution (compression)
                       stride=1, # the "step" size of the kernel
                       padding=0) # elements added to the input tensor boundaries to manipulate the output size

conv_output = conv_layer(train_sample[0]) # send the first training sample from the batch through the conv2d layer
print(f'Shape of sample before conv: {train_sample[0].shape}')
print(f'Shape of sample after conv: {conv_output.shape}') 
print(f'Sample after convolution:\n {conv_output}')

#### Note how the size of the images is compressed after the convolution. This is expected because we want to only get the most relevant information from the training sample, which could initially be a large tensor!. Also, we increased the channel size by quite a bit. Output channels can also be called **filters**. Under the hood, there is a **kernel** for each channel in the previous layer *for each neuron in the conv layer*. For example, there are 3 kernels per neuron in the first conv layer since the training sample has 3 channels. Since we defined the conv2d input layer to have 10 output channels, this corresponds to the *number of neurons in the layer*. With CNNs, the *learned weights by the model are stored in the kernels!* The kernels sweep the input image from the previous layer (called an **activation map**) and do element wise multiplication with the values in the input activation map of the corresponding channel and the learned weights in the kernel (there is a weight per element in each kernel). Note that for each neuron, the results of the convolution of each kernel with the corressponding activation map of the previous layer are *summed*, elementwise, along with the bias and then are passed along as an activation map for the next layer. The CNN Explainer website has a beautiful visual for this operation.

In [ ]:
# Now lets explore the other new layer, nn.MaxPool2d. Any time "pool" is in a name, it means compression.
# So the MaxPool2d layer is similar to the conv2d layer in that there is a kernel that sweeps along the
# input image, but instead of the convolution operation being done, the kernel selects the maximum value
# of the values that are "covered" by the kernel at a given step. So for a 2x2 kernel, the maxiumum value
# from a 2x2 grid is selected and added to the activation map. Note that MaxPool layers are not fully connected,
# they are one-to-one. These typically come after an activation function has been applied to the activation maps
# of the neurons in the previous conv layer. A 2x2 kernel size for MaxPool2d will cut the image size in half! (compression)
pool = nn.MaxPool2d(kernel_size=(2,2))
pool_output = pool(train_sample[0])
print(f'Shape of sample before MaxPool: {train_sample[0].shape}')
print(f'Shape of sample after MaxPool: {pool_output.shape}') 
print(f'Sample after Pooling:\n {pool_output}')

In [ ]:
# Now lets build the model! We'll be using the TinyVGG architecture while changing some params to fit
# the FashionMNIST data.
KERNEL_SIZE = 3
PADDING = 1
torch.manual_seed(SEED)

class FashionMNISTmodel_cnn(nn.Module):
    def __init__(self, in_channels, hidden_units, output_shape):
        super().__init__()
        # Lets define the first "block" from TinyVGG (Conv2d -> ReLU -> Conv2d -> ReLU -> MaxPool)
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=hidden_units, kernel_size=KERNEL_SIZE, stride=1, padding=PADDING), # padding keeps sample img size the same thru layer
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=KERNEL_SIZE, stride=1, padding=PADDING),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        # Lets define the second "block" from TinyVGG (Conv2d -> ReLU() -> Conv2d -> ReLU -> MaxPool)
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=KERNEL_SIZE, stride=1, padding=PADDING), # padding keeps sample img size the same thru layer
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=KERNEL_SIZE, stride=1, padding=PADDING),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        # Finally, lets define the last block, which flattens the activation maps after the final pooling layer and then does the classification
        self.block3 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=7*7*hidden_units, out_features=output_shape, bias=True)
        )
    def forward(self, x):
        return self.block3(self.block2(self.block1(x)))
    
fmnist_model_2_cnn = FashionMNISTmodel_cnn(in_channels=1, hidden_units=10, output_shape=len(train_data.classes))
fmnist_model_2_cnn

In [ ]:
# Now lets create an optimizer/loss function and run the train/test loop
loss_fn = nn.CrossEntropyLoss()
optimizer_v2 = torch.optim.SGD(params=fmnist_model_2_cnn.parameters(), lr=0.1)

EPOCHS = 5
train_time_mnn_start = timer()
for epoch in tqdm(range(EPOCHS)):
    print(f'Epoch: {epoch}\n-----------')
    train_step(
        fmnist_model_2_cnn,
        train_dloader,
        loss_fn,
        optimizer_v2,
        model_accuracy
    )
    test_step(
        fmnist_model_2_cnn,
        test_dloader,
        loss_fn,
        model_accuracy
    )
train_time_mnn_end = timer()
print(f'Total time to train: {total_time(train_time_mnn_start, train_time_mnn_end):.2f}s')

### The results above with the given seed are encouraging! The performance of the CNN is an improvement compared to the previous two models. Now lets explore a more useful way of determining the performance of the model by using a confusion matrix. This will give us a better idea of *how* the model is underperforming by showing us the classes that the model mis-classifies the most and which classes are "confused" the most.

In [ ]:
# First lets create a function that does inference based on the trained model
def make_predictions(model: nn.Module, samples: list):
    """
    Given a list of samples, returns a list of the probabilities for each class for each sample
    """
    model.eval()
    with torch.inference_mode():
        preds_list = [torch.softmax(model(x), dim=1) for x in tqdm(samples, desc="Making predictions...")] # apply softmax to logits of each sample in sample list
        return torch.cat(preds_list) # return as tensor instead of list


# Pick n random samples/labels from the test data
test_samples = []
test_labels = []

import random
for sample, label in random.sample(list(test_data), k=10): # random.sample samples k elements from the given population without replacement; returns list of samples.
    test_samples.append(sample)
    test_labels.append(label)

print(f'List size: {len(test_labels)}')
print(f'First sample shape from list: {test_samples[0].shape}')
print(f'First label from list: {test_labels[0]}')

# Now lets make the predictions and inspect
probs = make_predictions(fmnist_model_2_cnn, [x.unsqueeze(dim=0) for x in test_samples]) # need to add a dimension to the data to get it through the model.
print(f'probs shape: {probs.shape}')
print(f'probs length: {len(probs)}')
print(f'probs for first sample:\n {probs[0]}')

# Now lets collapse the tensors of probabilities to the label
preds = torch.argmax(probs.squeeze(), dim=1) # tensor is 10x1x10, squeeze out the unity dim. and apply argmax in the x direction for each sample (dim 1)
print(f'preds length: {len(preds)}')
print(f'label prediction for first sample: {preds[0]}')


In [ ]:
test_labels, preds # These two are very similar, the 7th sample has the incorrect prediction!

In [ ]:
# Now that we have a way to get predictions from the model using the test data, we can now create a confusion matrix.
# Pytorch already has a confusion matrix implementation that we can use and mlxtend is a package that plots the matrix
# in a useful way for us.
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

# Lets get logits from the FULL test dataset now and then send to argmax to get label predictions
preds = torch.argmax(make_predictions(fmnist_model_2_cnn, [X for X, _ in test_dloader]), dim=1) # get all the sample tensors from test_dloader and get the predictions.

# With the predictions on the entire test dataset, can now create and plot the confusion matrix
confmat = ConfusionMatrix(task='multiclass', num_classes=len(test_data.classes))
confmat_tensor = confmat(preds=preds, target=test_data.targets)

# Plot the confustion matrix with class labels
fig, ax = plot_confusion_matrix(
    confmat_tensor.numpy(), class_names=test_data.classes
)

### Looking at the confusion matrix, most predictions are accurate, but the model seems to get confused between the "t-shirt/top" and "shirt" classes and between the "shirt" and "coat" classes the most!